# RQ4

In [1]:
import pandas as pd   # importing pandas

### premier league's teams

In [3]:
teams = pd.read_json('C:\\Users\\Gabriele\\Desktop\\datasets\\teams.json')

for i in range(len(teams)):    
# drop national team
    if teams.loc[i, 'type'] != 'club':
        teams = teams.drop(i)         
# drop non premier's teams (swansea is in wales)
    elif teams.loc[i, 'area']['name'] != 'England' and teams.loc[i, 'area']['name'] != 'Wales':   
        teams = teams.drop(i)

teams = teams.reset_index(drop=True)


### premier league's players

In [4]:
players = pd.read_json('C:\\Users\\Gabriele\\Desktop\\datasets\\players.json')

teamsId = list(teams['wyId'])
# drop non premier league's players
for i in range(len(players)):  
    if players.loc[i, 'currentTeamId'] not in teamsId:
        players = players.drop(i)

players = players.reset_index(drop=True)


### number of ATTEMPTED passes for every players 

In [5]:
events_england = pd.read_json('C:\\Users\\Gabriele\\Desktop\\datasets\\events_England.json')

events_england = events_england[events_england['eventName'] == 'Pass']  # only 'pass' events
events_england = events_england.reset_index(drop=True)

# creating a dict with keys(player), value(number of attempted passes)
pass_att = {} 

for index1 in range(len(players)):
    count = events_england[events_england['playerId'] == players.loc[index1, 'wyId']] # all attempted passes of one player
    
    pass_att[players.loc[index1, 'shortName']] = len(count) # value of that player = number of attempted passes

# removing the players who didn't reach a reasonably number of passes
 
l_key = [] # list of keys to remove

for key, value in pass_att.items():  
    if value < 150:
        l_key.append(key)
        
# removing keys from the dict (values < 150 passes) = players who didn't reach a reasonably number of passes
for j in l_key:  
    pass_att.pop(j)   


### new dataframe without meaningless events

In [6]:
# removing from events_england the players removed before

for i in range(len(players)):
    if players.loc[i, 'shortName'] not in pass_att.keys():
        players = players.drop(i)

players = players.reset_index(drop=True)

# creating a dataframe of events without events of removed players 

for i in range(len(players)):
    if events_england.loc[i, 'playerId'] not in players['wyId']:
        events_england = events_england.drop(i)

events_england = events_england.reset_index(drop=True)


### number of COMPLETED passes for every players

In [7]:
# creating a dataframe with only the completed passes

d = {'id': 1801}  #  tag 'completed pass'

# creating a new column('status') which says if the pass is completed or not
l_tags = events_england['tags'] 
status = []
for i in range(len(l_tags)):

    if d not in l_tags[i]:
        status.append('no')
    else:
        status.append('completed')
        
# add the new column to events dataframe
events_england['status'] = status  

del events_england['tags']

# taking only the completed passes
com_pass = events_england[events_england['status'] == 'completed']


### percent of completed passes and first 10 most accurate players

In [8]:
# creating a dict with keys(player), value(number of completed passes)
pass_com = {} 

for index1 in range(len(players)):
    count1 = com_pass[com_pass['playerId'] == players.loc[index1, 'wyId']] # all completed passes of one player
    
    pass_com[players.loc[index1, 'shortName']] = len(count1) # value of that player = number of completed passes

# creating a dict with completed/attempted

for key, value in pass_com.items():
    
# change the values of pass_att with the ratio of completed / attempted
    pass_att[key] = (pass_com[key]/pass_att[key])*100  

pass_ratio = pass_att 

pass_ratio = sorted(pass_ratio.items(), key=lambda x: x[1], reverse=True) # sorting by value

print(pass_ratio[:10]) # print the 10 most accurate playera


[('J. Stones', 97.59136212624585), ('F. Delph', 94.50757575757575), ('V. Kompany', 94.32063263838964), ('A. Christensen', 94.02173913043478), ('P. Jones', 93.85052034058657), ('N. Otamendi', 93.75843454790824), ('\\u0130. G\\u00fcndo\\u011fan', 93.55555555555556), ('V. Lindel\\u00f6f', 93.55281207133059), ('M. Demb\\u00e9l\\u00e9', 93.26007326007326), ('Ederson', 93.24487334137515)]
